In [28]:
import import_ipynb
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
import lenet
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import KMNIST
from torchvision.datasets import ImageFolder
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time
# from lenet import Test

In [29]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
# load the KMNIST dataset
print("[INFO] loading the custom dataset...")
# trainData = KMNIST(root="data", train=True, download=True, transform=ToTensor())
# testData = KMNIST(root="data", train=False, download=True, transform=ToTensor())
transform = transforms.Compose([
    # resize
    #transforms.Resize(32),
    # center-crop
    #transforms.CenterCrop(32),
    # to-tensor
    transforms.ToTensor(),
    # normalize
    #transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
dataset = ImageFolder("images", transform)
trainDataset, valDataset, testDataset = random_split(dataset, [180, 30, 30])
# calculate the train/validation split
print("[INFO] generating the train/validation split...")
#numTrainSamples = int(len(trainData) * TRAIN_SPLIT)
#numValSamples = int(len(trainData) * VAL_SPLIT)
#(trainData, valData) = random_split(trainData,
#	[numTrainSamples, numValSamples],
#	generator=torch.Generator().manual_seed(42))

[INFO] loading the custom dataset...
[INFO] generating the train/validation split...


In [31]:
# initialize the train, validation, and test data loaders
trainDataLoader = DataLoader(trainDataset, shuffle=True,
	batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valDataset, batch_size=BATCH_SIZE)
testDataLoader = DataLoader(testDataset, batch_size=BATCH_SIZE)
# calculate steps per epoch for training and validation set
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
valSteps = len(valDataLoader.dataset) // BATCH_SIZE

In [32]:
model = lenet.Test()
print(model.print_animal())

this is an animal


In [35]:
# initialize the LeNet model
print("[INFO] initializing the LeNet model...")
model = lenet.LeNet(
	numChannels=3,
	classes=len(trainDataset.dataset.classes)).to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

[INFO] initializing the LeNet model...
[INFO] training the network...


In [36]:
# loop over our epochs
for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in trainDataLoader:
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (pred.argmax(1) == y).type(
			torch.float).sum().item()

	# switch off autograd for evaluation
	with torch.no_grad():
		# set the model in evaluation mode
		model.eval()
		# loop over the validation set
		for (x, y) in valDataLoader:
			# send the input to the device
			(x, y) = (x.to(device), y.to(device))
			# make the predictions and calculate the validation loss
			pred = model(x)
			totalValLoss += lossFn(pred, y)
			# calculate the number of correct predictions
			valCorrect += (pred.argmax(1) == y).type(
				torch.float).sum().item()

	# calculate the average training and validation loss
	avgTrainLoss = totalTrainLoss / trainSteps
	avgValLoss = totalValLoss / valSteps
	# calculate the training and validation accuracy
	trainCorrect = trainCorrect / len(trainDataLoader.dataset)
	valCorrect = valCorrect / len(valDataLoader.dataset)
	# update our training history
	H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
	H["train_acc"].append(trainCorrect)
	H["val_loss"].append(avgValLoss.cpu().detach().numpy())
	H["val_acc"].append(valCorrect)
	# print the model training and validation information
	print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
	print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(
		avgTrainLoss, trainCorrect))
	print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(
		avgValLoss, valCorrect))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x470450 and 800x500)

In [13]:
# finish measuring how long training took
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
	endTime - startTime))
# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
with torch.no_grad():
	# set the model in evaluation mode
	model.eval()
	
	# initialize a list to store our predictions
	preds = []
	# loop over the test set
	for (x, y) in testDataLoader:
		# send the input to the device
		x = x.to(device)
		# make the predictions and add them to the list
		pred = model(x)
		preds.extend(pred.argmax(axis=1).cpu().numpy())
# generate a classification report
print(classification_report(testDataset.targets.cpu().numpy(),
	np.array(preds), target_names=testDataset.classes))

[INFO] total time taken to train the model: 181.66s
[INFO] evaluating network...
              precision    recall  f1-score   support

           o       0.97      0.94      0.95      1000
          ki       0.97      0.92      0.94      1000
          su       0.92      0.92      0.92      1000
         tsu       0.96      0.97      0.97      1000
          na       0.94      0.93      0.93      1000
          ha       0.97      0.94      0.96      1000
          ma       0.91      0.97      0.94      1000
          ya       0.97      0.97      0.97      1000
          re       0.92      0.99      0.95      1000
          wo       0.98      0.94      0.96      1000

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000



In [16]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.plot(H["train_acc"], label="train_acc")
plt.plot(H["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.jpg")
# serialize the model to disk
torch.save(model, "model.h5")